In [1]:
import cv2
import torch

import torch
import torch.nn as nn
import numpy as np
import torchvision
from PIL import Image
from matplotlib import cm
import torch.nn.functional as F

# import confusion_matrix
import ckp_attn_model
from ckp_attn_model import BaselineModel
import data_loader
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, auc

In [2]:
## fer map
emotion_dict = {0: "Anger", 1: "Contempt", 2: "Disgust", 3: "Fear", 4: "Happy", 5: "Sadness", 6: "Surprise"}


In [3]:
model = BaselineModel(num_blocks=3)
model.load_state_dict(torch.load('models/fer_cnn_ckp_attention_best_model.pt', map_location=torch.device('cpu')))
model.eval()


# loader = data_loader.get_test_loader() 
# all_preds = []
# all_labels = []

# with torch.no_grad():
#     for batch_num, (feats, labels) in enumerate(loader):
#         outputs = model(feats)
        
#         _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
#         pred_labels = pred_labels.view(-1)
        
#         all_preds.append(pred_labels[0].item())
#         all_labels.append(labels[0].item())
        

# metrics.confusion_matrix(all_labels, all_preds)


# print(all_preds)
# print(all_labels)

BaselineModel(
  (conv1): ConvBlock(
    (block): Sequential(
      (0): Conv2d(1, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (conv2): ConvBlock(
    (block): Sequential(
      (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): ConvBlock(
    (block): Sequential(
      (0): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (conv4): ConvBlock(
    (block): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05

In [4]:
# # img = Image.open('imgs/surprise.png')
# # img_pil = torchvision.transforms.Resize((48,48))(img)
# # img = torchvision.transforms.ToTensor()(img_pil)
# # img = torchvision.transforms.Normalize(mean=[0.485], std=[0.229])(img)
# # img = img.unsqueeze(dim=0)
# # prhttp://localhost:8888/notebooks/DL/project/CK%2B/pythons/video%20code/ckp.ipynb#int("img ", img)

# # # img_pil.show()


# frame = cv2.imread('imgs/surprise.png')
# print("frame type: ", type(frame))
# test = Image.fromarray(frame)
# test_img_pil = torchvision.transforms.Resize((48,48))(test)
# test = torchvision.transforms.ToTensor()(test_img_pil)
# test = torchvision.transforms.Normalize(mean=[0.485], std=[0.229])(test)
# print("test: ", test)


# img = Image.open('imgs/surprise.png')
# img = Image.open('imgs/surprise.png')
# img_pil = torchvision.transforms.Resize((48,48))(img)
# img = torchvision.transforms.ToTensor()(img_pil)
# img = torchvision.transforms.Normalize(mean=[0.485], std=[0.229])(img)
# print("IMage: ", img)

# # print("img: ", type(img))
    



In [ ]:
# # Load the cascade
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Read the input image
frame = cv2.imread('imgs/random_smile.png')

# Convert into grayscale
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
print("frame shape: ", frame.shape)

# Detect faces
faces = face_cascade.detectMultiScale(gray, 1.1, 4)


# Draw the rectangle around each face
for (x, y, w, h) in faces:
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
    roi_gray = gray[y:y + h, x:x + w]
    
    
    cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), 0), 0)
    cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
    
#     print("img shape: ", cropped_img.shape)
#     print(type(cropped_img))

    
#     img = Image.open('imgs/surprise.png')
    img = Image.fromarray(frame)
    img_pil = torchvision.transforms.Resize((48,48))(img)
    img = torchvision.transforms.ToTensor()(img_pil)
    if img.shape[0] == 3:
        img = torchvision.transforms.Grayscale(num_output_channels=1)(img_pil)
        img = torchvision.transforms.ToTensor()(img)
    
    
    img = torchvision.transforms.Normalize(mean=[0.485], std=[0.229])(img)
    img = img.unsqueeze(dim=0)
    print("img ", img.shape)
    prediction = model(img)
        
    print(prediction)
    print(torch.argmax(prediction).item())
#     print(emotion_dict)
    cv2.putText(frame, emotion_dict[int(torch.argmax(prediction).item())], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)


    
# Display
cv2.imshow('frame', frame)
cv2.waitKey(0)
cv2.destroyWindow('frame')

('frame shape: ', (670, 464, 3))
('img ', torch.Size([1, 1, 48, 48]))
tensor([[-0.5199,  3.4965, -5.7553,  4.2202, -3.4968, -3.0499, -3.5751]],
       grad_fn=<AddmmBackward>)
3


In [ ]:
# # # To capture video from webcam. 
# cap = cv2.VideoCapture(0)
# # # To use a video file as input 
# # cap = cv2.VideoCapture('filename.mp4')

# # # Load the cascade
# face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


# while True:
#     # Read the frame
#     ret, frame = cap.read()
    
#     # Convert to grayscale
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
#     # Detect the faces
#     faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
#     # Draw the rectangle around each face
#     for (x, y, w, h) in faces:
#         cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
#         roi_gray = gray[y:y + h, x:x + w]
#         cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), 0), 0)

#         cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)

# #         cropped_img = torch.from_numpy(cropped_img)
# #         cropped_img = cropped_img.float()

#         img = Image.fromarray(frame)
#         img_pil = torchvision.transforms.Resize((48,48))(img)
#         img = torchvision.transforms.ToTensor()(img_pil)
#         if img.shape[0] == 3:
#             img = torchvision.transforms.Grayscale(num_output_channels=1)(img_pil)
# #             img.show()
#             img = torchvision.transforms.ToTensor()(img)

#         img = torchvision.transforms.Normalize(mean=[0.485], std=[0.229])(img)
#         img = img.unsqueeze(dim=0)
#         prediction = model(img)
        
#         print(prediction)
#         cv2.putText(frame, emotion_dict[int(torch.argmax(prediction).item())], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)

#     # Display
#     cv2.imshow('frame', frame)
    
#     # Stop if q key is pressed
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
        
# # Release the VideoCapture object
# cap.release()
# cv2.destroyAllWindows()